In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def load_data(l):
    data=np.array(pd.read_csv(l,delimiter=",",skiprows=1))
    X = data[:,1:]
    Y = data[:,0]
    return (X,Y)
def split(X,Y,s):
    m=X.shape[0]
    a=math.ceil(s*m)
    return(X[:a],Y[:a],X[a:],Y[a:])

In [ ]:
def sigmoid(X,w,b):
    return  1 / (1 + np.exp(-(X @ w + b)))    
def loss(g,y):
    m = X.shape[0]
    loss = (-1 / m) * np.sum(y * np.log(g+1e-20))
    return loss
def gradient_descent(one_hot_encoding,sigmoid,X,Y,classes,alpha,steps):
    m, n = X.shape
    cost = []
    w = np.zeros((n,classes))
    b = np.zeros(classes)
    y = one_hot_encoding(X,Y,classes)
    i_list = []
    for i in range(steps+1):
        i_list.append(i)
        g = sigmoid(X,w,b)
        error = g - y
        w -= alpha * ((X.T @ error) / m)
        b -= alpha * (np.sum(error,axis=0) / m)
        cost.append(loss(g,y))
        if i % math.ceil(steps/10) == 0:
            print(f"Iteration {i:9d}, Cost: {cost[i]}")
    return w,b, cost,i_list

def one_hot_encoding(X,Y,classes):
    m, n = X.shape
    y_he = np.zeros((m,classes))
    for k in range(classes):
        y_he[:,k] = np.where(Y == k, 1, 0)    
    print("1-hot-encoding done")
    return y_he

def predict_logistic(X, w, b):
    m = X.shape[0]
    probabilities = 1 / (1 + np.exp(-(X @ w + b)+1e-100))
    predictions = np.argmax(probabilities, axis=1) 
    return predictions

In [ ]:
def f1score(classes, y_train,y_predicted):
    con_matrix = np.zeros((classes,classes),dtype=int)
    for i, k in zip(y_train, y_predicted):
        con_matrix[int(i),int(k)] += 1
    df = pd.DataFrame(con_matrix)
    precision = np.zeros(classes)
    recall = np.zeros(classes)
    f1 = np.zeros(classes)
    for i in range(classes):
        t_p = con_matrix[i, i]
        f_p = np.sum(con_matrix[:, i]) - t_p
        f_n = np.sum(con_matrix[i, :]) - t_p
        precision[i] = t_p / (t_p + f_p) if (t_p + f_p) != 0 else 0
        recall[i] = t_p / (t_p + f_n) if (t_p + f_n) != 0 else 0
        f1[i] = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i]) if (precision[i] + recall[i]) != 0 else 0
    f1score = np.mean(f1)
    return f1score, df

def plot(cost,i_list,alpha,iteration):
    plt.plot(i_list,cost,c="r",label =f"cost \n alpha={alpha} \n No. of iteration ={iteration}")
    plt.xlabel(f"iteration")
    plt.ylabel("cost")
    plt.legend()
    plt.show()
    
def plot_data(X, y,y_pred):
    marker_styles = ['o', 's', '^', 'v', 'D','x','+','*','<','>']
    fig,axs = plt.subplots(5, 2, figsize=(12, 20), sharey=True)
    for i in range(classes):
        cls1 = y[:] == i
        cls2 = y_pred[:] == i
        label_ = "cls " + str(i)
        axs[i//2,i%2].scatter(X[cls1, 428], X[cls1, 427],c="r", label="true")
        axs[i//2,i%2].scatter(X[cls2, 428], X[cls2, 427], marker=marker_styles[i],c="b",s=10,label="predicted")
        axs[i//2,i%2].legend(loc="upper right")
        axs[i//2,i%2].set_xlabel(f'Feature {i}')
    plt.ylabel('class 2')  
    plt.show()


In [ ]:
X,Y = load_data("Classification_train.csv")
X_new=X/255
test_size = 0.7
X_train,Y_train,X_cv,Y_cv = split(X_new,Y,test_size)

In [ ]:
print(f"max in original :{np.max(X)}, min in original :{np.min(X)}")   
print(f"max in normalized :{np.max(X_new)}, min in original :{np.min(X_new)}")   

In [ ]:
classes = 10
alpha = 0.9
iteration = 10000
print(f"Alpha: {alpha}, No. of iteration: {iteration}")
w_final, b_final, cost_train,i_list = gradient_descent(one_hot_encoding,sigmoid,X_train, Y_train, classes, alpha, iteration)

In [ ]:
plot(cost_train,i_list,alpha,iteration)

In [ ]:
#cross validating
y_cv_n = one_hot_encoding(X_cv,Y_cv,classes)
print(pd.DataFrame(y_cv_n[:10]))

g = sigmoid(X_cv,w_final,b_final)
cost_cv = loss(g,y_cv_n)
print(f"cost after cross validating: {cost_cv}")

p_cv = predict_logistic(X_cv, w_final, b_final)
print('Train Accuracy: %f'%(np.mean(p_cv == Y_cv) * 100))

In [ ]:
f1_score, confusion_matrix = f1score(classes,Y_cv,p_cv)
print(f"f1score: {f1_score}")
print()
print("confusion_matrix")
confusion_matrix

In [ ]:
plot_data(X_cv, Y_cv,p_cv)

In [ ]:
#testing
X_test,Y_id=load_data("Classification_test.csv")
X=X_test
p_test = predict_logistic(X_test, w_final, b_final)
np.savetxt('logistic_test_prediction.csv', p_test, delimiter=',', header='Test target', comments='')